<a href="https://colab.research.google.com/github/Heity94/TWSM_Lab/blob/main/Project/Notebooks/PH_TrainTestSplit_Ontology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Declare whether you are on Colab or local
colab = True

In [2]:
if colab==True:
  
  #Mount drive
  from google.colab import drive
  drive.mount('/content/drive')

  #set path to data in Google Drive
  data_path = "/content/drive/MyDrive/2022_Analytics Lab Student Projects/Data/All Topics"

  #install required packages
  !pip install duckdb
  #!pip install -U sentence-transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Import sentence_transformers 
# from sentence_transformers import SentenceTransformer, util

In [4]:
#Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import duckdb
import sqlalchemy
#%load_ext sql

# Train-validation-test split the ontology

### Load data

We have multiple data sources which we could use to "split" the ontoloogy. 

#### Ontology + synonyms

This list "only" contains the main entity_id and the synonym. Will be relevant as soon as we have a list of main entities which we want to exclude from the train dataset .

In [5]:
if colab==True:
  ontology_syn = pd.read_csv(data_path+"/Synonyms in Ontology/isontology.csv", index_col=0)
else:
  ontology_syn = pd.read_csv("../Data/SynonymsinOntology/isontology.csv", index_col=0)
ontology_syn.shape

(397942, 4)

In [6]:
ontology_syn.head()

,entity_id,category,label,synonym
0,IS topic,domain specific entity,TOPIC,IS topic
1,IS topic,domain specific entity,TOPIC,IS topics
2,IS sourcing,domain specific entity,TOPIC,IS / IT sourcing
3,IS sourcing,domain specific entity,TOPIC,tool sourcing
4,IS sourcing,domain specific entity,TOPIC,software sourcings


##### Downsampling entities with more than 100 synonyms

In [7]:
## Check number of synonyms per entity_id
#ent_sy_group = ontology_syn.groupby("entity_id")[["synonym"]].nunique().sort_values(by="synonym", ascending=False)

In [8]:
# # Downsample entities with more than 100 synonyms to a 100 synoyms (but always include "main entity_id" in subsample)
# idx = ent_sy_group[ent_sy_group["synonym"]>100].index.tolist()

# dfs = []
# for entity in idx:
#   t1 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym==entity)]
#   t2 = ontology_syn[(ontology_syn.entity_id==entity)&(ontology_syn.synonym!=entity)].sample(99)
#   dfs.append(pd.concat([t1,t2]))
# dfs.append(ontology_syn.loc[~ontology_syn["entity_id"].isin(idx)]) # Append dataframe for all entities with less then 100 synonyms to list

# # Combine all dataframes 
# ontology_syn_ds = pd.concat(dfs).reset_index(drop=True)

In [9]:
#Show synonyms in ontology which appear in more than 1 entity
#ontology_syn_ds[ontology_syn_ds.synonym.duplicated(keep=False)].sort_values(by="synonym")

In [10]:
#List of unique entities from ontology with synonyms with downsampling (apparently there are some duplicates in there?!)
#entities_syn_unique_ds = ontology_syn_ds.synonym.unique().tolist()
#ontology_syn_ds_unique = ontology_syn_ds.drop_duplicates(subset="synonym")

#### Ontology.json (only structure of ontology with "main" entity ids (without synonmys))

I did not manage yet to flatten the json file, therefore I used the entities table from the duckdb instead (see next section) 

In [11]:
if colab==True:
  ontology_json = pd.read_json(data_path+"/Ontology/isontology.json")#, orient="values")#, index_col=0)

In [12]:
ontology_json.head()

,root
0,{'IS topic': [{'IS sourcing': [{'outsourcing':...
1,{'research paradigm': [{'positivism': ['logica...
2,"{'model': ['taxonomy', 'framework', {'research..."
3,"{'economic sector': ['private household', {'go..."
4,{'company type': [{'small and mid-size enterpr...


#### Entities (Sentence_id + tagged entity_id) by Rolands script

In [13]:
# Note that DBeaver must be closed, before you can run this command!
if colab==True:
  con = duckdb.connect(database=data_path+'/Extracted Information of Corpus/isrecon.duckdb', read_only=False)
else:
  con = duckdb.connect(database='../Data/ExtractedInformationofCorpus/isrecon.duckdb', read_only=False)

Lets load all the entities from the duckdb

In [14]:
# Load first article as df
entities = con.execute("""
                       SELECT article_id, sentence_id, ent_id, level_1, level_2, level_3, level_4, level_5, level_6, level_7, level_8, level_9, level_10,
                       FROM entities e
                       """).fetchdf()

entities.shape

(1797740, 13)

In [15]:
# Main entities of first level of ontology
entities.level_1.unique()

array(['domain specific entity', 'theoretical entity',
       'methodological entity'], dtype=object)

In [96]:
# Distribution of entities from database accross first and second level
entities.groupby(["level_1", "level_2"])[["ent_id"]].aggregate(["count", "nunique"])

ent_id        
                                              count nunique
level_1                level_2                             
domain specific entity IS technology         463211     475
                       IS topic              461262     416
                       economic sector       102057     234
                       study object           61799      21
methodological entity  data analysis method   73002     292
                       research method       257820     272
                       sampling                6148      27
                       statistical metric     78673      87
                       validity               11357      25
theoretical entity     level of analysis      12064      13
                       model                 196827      31
                       research paradigm      13429      14
                       theory                 60091     223

## `Train-val-test` split of the existing ontology

In order to determine how well our model/embeddings are in identifing new entities or synonyms for existing entities we have to create a `train-val-test split` version of the existing ontology. Meaning we have to select some entities which will be removed from the ontology for training. 


- We will remove different nodes across all main topics and levels of the ontology to assess how our model is handling with missing nodes from different "levels"
- To determine which nodes to include we used the `ontology.json` file since here the hierarchy is already visually included, so it was easy to define some random nodes (and child nodes)

**We selcted the following entities as part of the validattion-test set:**

|    | level_1                | level_2              | entity_id_to_remove                 |   has_child_nodes |
|---:|:-----------------------|:---------------------|:------------------------------------|------------------:|
|  0 | domain specific entity | IS topic             | business process outsourcing        |                 1 |
|  1 | domain specific entity | IS topic             | cybersecurity behavior              |                 1 |
|  2 | domain specific entity | IS topic             | user interface                      |                 1 |
|  3 | domain specific entity | IS topic             | IT planning                         |                 1 |
|  4 | domain specific entity | IS topic             | business process management         |                 1 |
|  5 | domain specific entity | IS topic             | sharing platform                    |                 1 |
|  6 | domain specific entity | IS topic             | software license                    |                 1 |
|  7 | domain specific entity | IS topic             | IT recruiting                       |                 0 |
|  8 | domain specific entity | IS technology        | enterprise service bus              |                 0 |
|  9 | domain specific entity | IS technology        | online marketing                    |                 1 |
| 10 | domain specific entity | IS technology        | video game                          |                 1 |
| 11 | domain specific entity | IS technology        | optical storage device              |                 1 |
| 12 | domain specific entity | IS technology        | recommender system                  |                 1 |
| 13 | domain specific entity | IS technology        | digital asset management            |                 0 |
| 14 | domain specific entity | IS technology        | high level language                 |                 1 |
| 15 | domain specific entity | Economic sector      | forest industry                     |                 1 |
| 16 | domain specific entity | Economic sector      | hedge fund                          |                 0 |
| 17 | domain specific entity | Economic sector      | structural engineering              |                 1 |
| 18 | domain specific entity | Economic sector      | research institute                  |                 0 |
| 19 | domain specific entity | Economic sector      | private household                   |                 0 |
| 20 | domain specific entity | Economic sector      | plastics industry                   |                 1 |
| 21 | domain specific entity | study object         | group participant                   |                 0 |
| 22 | domain specific entity | study object         | startup                             |                 0 |
| 23 | domain specific entity | study object         | microenterprise                     |                 0 |
| 24 | methodological entity  | research method      | pilot experiment                    |                 0 |
| 25 | methodological entity  | research method      | completely randomized design        |                 0 |
| 26 | methodological entity  | research method      | pilot survey                        |                 0 |
| 27 | methodological entity  | research method      | action research                     |                 1 |
| 28 | methodological entity  | research method      | delphi study                        |                 0 |
| 29 | methodological entity  | research method      | Markov model                        |                 1 |
| 30 | methodological entity  | research method      | database modeling                   |                 1 |
| 31 | methodological entity  | data analysis method | constant comparative analysis       |                 0 |
| 32 | methodological entity  | data analysis method | ensemble learning                   |                 1 |
| 33 | methodological entity  | data analysis method | sentiment analysis                  |                 1 |
| 34 | methodological entity  | data analysis method | autoregressive moving average model |                 0 |
| 35 | methodological entity  | data analysis method | positive correlation                |                 0 |
| 36 | methodological entity  | data analysis method | recurrent neural network            |                 1 |
| 37 | methodological entity  | data analysis method | exploratory factor analysis         |                 0 |
| 38 | methodological entity  | data analysis method | confirmatory factor analysis        |                 0 |
| 39 | methodological entity  | data analysis method | survival function                   |                 1 |
| 40 | methodological entity  | data analysis method | p hacking                           |                 0 |
| 41 | methodological entity  | statistical metric   | Chi square metric                   |                 0 |
| 42 | methodological entity  | statistical metric   | confusion matrix                    |                 0 |
| 43 | methodological entity  | statistical metric   | contingency table                   |                 0 |
| 44 | methodological entity  | validity             | formative validity                  |                 0 |
| 45 | methodological entity  | validity             | criterion validity                  |                 1 |
| 46 | methodological entity  | sampling             | oversampling                        |                 1 |
| 47 | methodological entity  | sampling             | quota sampling                      |                 0 |
| 48 | theoretical entity     | level of analysis    | micro level                         |                 1 |
| 49 | theoretical entity     | level of analysis    | country level                       |                 0 |
| 50 | theoretical entity     | model                | causal model                        |                 0 |
| 51 | theoretical entity     | model                | control variable                    |                 0 |
| 52 | theoretical entity     | model                | taxonomy                            |                 0 |
| 53 | theoretical entity     | model                | theoretical link                    |                 1 |
| 54 | theoretical entity     | model                | control variable                    |                 0 |
| 55 | theoretical entity     | model                | null hypothesis                     |                 0 |
| 56 | theoretical entity     | model                | nomological network                 |                 0 |
| 57 | theoretical entity     | research paradigm    | interpretivism                      |                 0 |
| 58 | theoretical entity     | research paradigm    | realism                             |                 1 |
| 59 | theoretical entity     | theory               | IT security theory                  |                 1 |
| 60 | theoretical entity     | theory               | theory of task technology fit       |                 0 |
| 61 | theoretical entity     | theory               | personality theory                  |                 1 |
| 62 | theoretical entity     | theory               | equity theory                       |                 0 |
| 63 | theoretical entity     | theory               | two-factor theory                   |                 0 |
| 64 | theoretical entity     | theory               | personality theory                  |                 1 |
| 65 | theoretical entity     | theory               | uncanny valley theory               |                 0 |

We also stored the list above in an Excel format for further processing

In [146]:
data_path_group = data_path[:-10]+"Topic 1/Data_Team1/" # create new data path to access files created by Team1

entities_to_removed = pd.read_excel(data_path_group+"ent_to_remove_from_ontology.xlsx") #load excel list as dataframe
entities_to_removed["entity_id_to_remove"] = entities_to_removed.entity_id_to_remove.str.strip() #remove whitespace

In [147]:
entities_to_removed.head(2)

,level_1,level_2,entity_id_to_remove,has_child_nodes
0,domain specific entity,IS topic,business process outsourcing,1.0
1,domain specific entity,IS topic,cybersecurity behavior,1.0


Now we want to further enhance this dataframe to also include the entity level as well as the level and the name of the parent entitiy

In [150]:
# Create list of entites which should be removed
ent_to_rmv_list = entities_to_removed.entity_id_to_remove.to_list()

In [151]:
col_names = ['ent_id', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6', 'level_7', 'level_8', 'level_9', 'level_10']
entities[col_names].sample(1)

,ent_id,level_1,level_2,level_3,level_4,level_5,level_6,level_7,level_8,level_9,level_10
1722670,IS technology,domain specific entity,IS technology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Each entry in the entity table includes information about the matched entity as well as the "position" within the ontology

In [152]:
# Define function to get last index which is not NaN => The level of the entity within the ontology
def get_last(a):
  '''Get last index of list which is not NaN'''
  for i, e in enumerate(reversed(a)):
    if e is not np.nan:
      return len(a) - i -1
  return -1

In [153]:
# Function to take a dataframe and create dictonary of level of entity, level of parent entity and name of parent entity
def get_idx_par_ent(df):
  '''Get level of entity, level of parent entity and name of parent entity'''
  ent_lvls = df.values.flatten().tolist() 
  lvl_ent = get_last(ent_lvls)
  lvl_par_ent = lvl_ent-1
  par_ent = ent_lvls[lvl_par_ent]

  return {"entity_id":ent_lvls[0], "lvl_ent":lvl_ent, "lvl_par_ent":lvl_par_ent, "par_ent":par_ent}

In [154]:
col_names = ['ent_id', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'level_6', 'level_7', 'level_8', 'level_9', 'level_10']

# Create df which includes one row per entity which should be removed (since we only care about the position in the ontology)
ent_rmv_uniq = entities[entities.ent_id.isin(entities_to_removed.entity_id_to_remove.to_list())].drop_duplicates("ent_id")[col_names]

# Apply function to get level information of each entity and store result as dataframe
ent_rmv_uniq_df = ent_rmv_uniq.apply(lambda x: get_idx_par_ent(x), axis=1, result_type='expand')

In [155]:
# Merge resulting dataframe onto existing entities_to_removed df
entities_to_removed = entities_to_removed.merge(ent_rmv_uniq_df ,left_on="entity_id_to_remove", 
                                                right_on="entity_id", how="left").drop(columns="entity_id")

In [156]:
# Check that we found a parent entity from each entity in the list
entities_to_removed[entities_to_removed.lvl_par_ent.isna()]

,level_1,level_2,entity_id_to_remove,has_child_nodes,lvl_ent,lvl_par_ent,par_ent


In [157]:
# Chck results visually
entities_to_removed.sample(2)

,level_1,level_2,entity_id_to_remove,has_child_nodes,lvl_ent,lvl_par_ent,par_ent
32,methodological entity,data analysis method,ensemble learning,1.0,8,7,classification method
63,theoretical entity,theory,two-factor theory,0.0,5,4,theory of work design


Now we want to even further enhance this dataframe to also include: 
- a list of different entities which belong to the specific parent node
- a list of sentences in which the entity occurs
- a list of articles in which the entity occurs
- also we want the number of occurences for all three points from above

In [158]:
tmp_list=[] # List of entities and child nodes which have to be removed from train ontology 

for entity in ent_to_rmv_list:
  
  #print(entity, (entities.values == entity).any(1).astype(int).max()) #to visually check if the random picked nodes are even included in entities database (so if there has been a match yet)

  tmp_ = entities[entities.values == entity][["ent_id", "article_id", "sentence_id"]].apply(lambda x: x.unique().tolist()) #get all the unique values for all three columns
  df_tmp = pd.DataFrame(tmp_).T #store results as df'
  df_tmp.columns=["ent_list", "articles_list", "sentences_list"] #rename columns
  tmp_list.append(df_tmp) #add df to list

# Add list of children per entity, list of articles and sentences as new column to dataframe
entities_to_removed = pd.concat((entities_to_removed, pd.concat(tmp_list).reset_index(drop=True)), axis=1)

# add number of occurences as separate columns
entities_to_removed[["num_ent", "num_articles", "num_sentences"]] = entities_to_removed[["ent_list", "articles_list", "sentences_list"]].applymap(lambda x: len(x))

In [159]:
entities_to_removed.sample(3)

,level_1,level_2,entity_id_to_remove,has_child_nodes,lvl_ent,lvl_par_ent,par_ent,ent_list,articles_list,sentences_list,num_ent,num_articles,num_sentences
63,theoretical entity,theory,two-factor theory,0.0,5,4,theory of work design,[two-factor theory],"[4276, 2496, 1062, 2751, 5122, 2079, 4040, 558...","[4276_5888_5913, 2496_5155_5164, 2496_5620_563...",1,9,48
35,methodological entity,data analysis method,positive correlation,0.0,6,5,correlation direction,[positive correlation],"[191, 2657, 3692, 4062, 654, 6328, 4259, 3209,...","[191_1683_1705, 2657_9769_9804, 3692_9326_9359...",1,289,427
12,domain specific entity,IS technology,recommender system,1.0,5,4,analytical information system,"[recommender system, collaborative filtering, ...","[1838, 5414, 5415, 5604, 1623, 2656, 2657, 439...","[1838_2291_2339, 5414_13785_13803, 5415_10502_...",3,311,2680


Now we can create a list of all entities and child entities which we then use to filter the entity dataframe 

In [160]:
#flatten list of entities which need to be removed 
entities_to_remove_cmpl = [x for xs in entities_to_removed.ent_list.to_list() for x in xs] # add all parent (and child) entities to a list which then can be excluded from the train dataframe

In [161]:
# Check number of entities which need to be removed from train set
len(entities_to_remove_cmpl)

156

In [214]:
# Lets check the distributiton of the selcted entities accross the ontology
entities_to_removed.groupby(["level_1", "level_2", "lvl_ent"]).aggregate({'num_ent' :["sum"], "entity_id_to_remove":["count"]})

num_ent  \
                                                        sum   
level_1                level_2              lvl_ent           
domain specific entity Economic sector      3             1   
                                            4            10   
                                            5             2   
                       IS technology        4            16   
                                            5            12   
                                            7             4   
                       IS topic             3            10   
                                            4            10   
                                            5             9   
                                            6             6   
                       study object         4             2   
                                            5             1   
methodological entity  data analysis method 4             1   
                                            5             6   
                                            6             1   
                                            7             3   
                                            8             8   
                                            9             2   
                       research method      5             3   
                                            6            11   
                                            7             2   
                       sampling             4             3   
                       statistical metric   3             2   
                                            4             1   
                       validity             4             1   
                                            5             3   
theoretical entity     level of analysis    3             3   
                                            4             1   
                       model                3             2   
                                            4             1   
                                            5             4   
                                            6             1   
                       research paradigm    3             2   
                                            4             1   
                       theory               4            10   
                                            5             1   

                                                    entity_id_to_remove  
                                                                  count  
level_1                level_2              lvl_ent                      
domain specific entity Economic sector      3                         1  
                                            4                         3  
                                            5                         2  
                       IS technology        4                         2  
                                            5                         4  
                                            7                         1  
                       IS topic             3                         1  
                                            4                         3  
                                            5                         3  
                                            6                         1  
                       study object         4                         2  
                                            5                         1  
methodological entity  data analysis method 4                         1  
                                            5                         3  
                                            6                         1  
                                            7                         3  
                                            8                         1  
                                            9                         1  
                       research method  

In [162]:
# Store final entities_to_remove df as .csv
#entities_to_removed.to_csv(data_path_group+"ent_to_remove_from_ontology_final.csv")

### Create train dataframe of ontology

In [163]:
# Create train dataframe which does NOT contain any of the entities and porential children
entities_train = entities[~entities.ent_id.isin(entities_to_remove_cmpl)]
entities_train.shape

(1709078, 13)

In [164]:
# Store train df as .csv
#entities_train.to_csv(data_path_group+"ontology_train.csv")

### Create validation & test dataframe of ontology

For the validation and test set we need to further split the `entities_to_removed` dataframe

In [169]:
# Distribution of entities from val_test split accross first and second level
entities[entities.ent_id.isin(entities_to_remove_cmpl)].groupby(["level_1", "level_2"])[["ent_id"]].aggregate(["count", "nunique"])

ent_id        
                                             count nunique
level_1                level_2                            
domain specific entity IS technology         12211      32
                       IS topic              29120      35
                       economic sector        2960      13
                       study object           7644       3
methodological entity  data analysis method   2733      20
                       research method        4737      16
                       sampling                 39       3
                       statistical metric     2833       4
                       validity                447       4
theoretical entity     level of analysis      5464       4
                       model                 15619       7
                       research paradigm      3615       3
                       theory                 1240       9

##### Test dataframe

In [198]:
# Select every 3rd row from the entities removed as part of the test set
test_ent_df = entities_to_removed.sort_values(["level_1", "level_2"]).reset_index(drop=True).iloc[::3, :]

In [202]:
test_entities_list = [x for xs in test_ent_df.ent_list.to_list() for x in xs] 

In [203]:
# Create train dataframe which does NOT contain any of the entities and porential children
entities_test = entities[entities.ent_id.isin(test_entities_list)]
entities_test.shape

(29080, 13)

In [209]:
# Store test df as .csv
#entities_test.to_csv(data_path_group+"ontology_test.csv")

##### Validation dataframe

In [206]:
# Select all rows/ entities which have not been selected for the test set 
val_ent_df = entities_to_removed[~entities_to_removed.entity_id_to_remove.isin(test_entities_list)]

In [207]:
# flatten list
val_entities_list = [x for xs in val_ent_df.ent_list.to_list() for x in xs] 

In [208]:
# Create train dataframe which does NOT contain any of the entities and porential children
entities_val = entities[entities.ent_id.isin(val_entities_list)]
entities_val.shape

(59582, 13)

In [210]:
# Store test df as .csv
# entities_val.to_csv(data_path_group+"ontology_val.csv")